In [11]:
s = "hello! こんにちは!"
enc = s.encode("utf-8")
enc_s = list(enc)
print(enc_s)

[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 33]


In [14]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

In [18]:
decode_utf8_bytes_to_str_wrong("hello! こんにちは!".encode("utf-8"))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: unexpected end of data

In [60]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [24]:
import regex as re
print(re.findall(PAT, "t'is is good for him to it's very as."))

['t', "'", 'is', ' is', ' good', ' for', ' him', ' to', ' it', "'s", ' very', ' as', '.']


In [56]:
text = """low low low low low lower lower widest widest widest newest newest newest newest newest newest"""
# es = re.findall(PAT, text)
# es

In [117]:
from typing import Dict, Any
enc : dict[tuple[bytes], int] = {}


In [118]:
# encs = [tuple(k.group(0).encode("utf-8")) for k in re.finditer(PAT, text)]
encs = [tuple(bytes([b]) for b in k.encode("utf-8")) for k in text.split(" ")]
# encs

In [119]:
for e in encs:
    enc[e] = enc.get(e, 0) + 1
enc

{(b'l', b'o', b'w'): 5,
 (b'l', b'o', b'w', b'e', b'r'): 2,
 (b'w', b'i', b'd', b'e', b's', b't'): 3,
 (b'n', b'e', b'w', b'e', b's', b't'): 6}

In [120]:
d = {}
for e in enc:
    for i, j in zip(e[:-1], e[1:]):
        d[(i, j)] = d.get((i, j), 0) + enc[e]

In [121]:
s = max(d, key=lambda x: (d[x], x))
s

(b's', b't')

In [123]:
#build new encoding now
new_enc = {}
merges = [] 
merges.append(s)
for tok, freq in enc.items():
    new_tok = []
    i = 0
    while i < len(tok):
        if i < len(tok) - 1 and (tok[i], tok[i + 1]) == s:
            new_tok.append(tok[i] + tok[i+1])
            i += 2
        else:
            new_tok.append(tok[i])
            i += 1
    new_enc[tuple(new_tok)] = new_enc.get(tuple(new_tok), 0) + freq
new_enc

{(b'l', b'o', b'w'): 5,
 (b'l', b'o', b'w', b'e', b'r'): 2,
 (b'w', b'i', b'd', b'e', b'st'): 3,
 (b'n', b'e', b'w', b'e', b'st'): 6}

In [141]:
text = """Hello, world! I'm testing GPT-2's BPE tokenizer.
Don't split wrongly: numbers like 123, punctuation!!! or spaces   matter.
"""

In [149]:
merges = []
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
# encs = [tuple(bytes([b]) for b in k.encode("utf-8")) for k in text.split(" ")]
encs = [tuple(bytes([b]) for b in k.group(0).encode("utf-8")) for k in re.finditer(PAT, text)]
enc = {}
for e in encs:
    enc[e] = enc.get(e, 0) + 1
num_merges = 6
for _ in range(num_merges):
    d = {}
    for e in enc:
        for i, j in zip(e[:-1], e[1:]):
            d[(i, j)] = d.get((i, j), 0) + enc[e]
    if not d:
        break
    s = max(d, key=lambda x: (d[x], x))
    # s = max(d, key = d.get)
    merges.append(s)
    new_enc = {}
    for tok, freq in enc.items():
        new_tok = []
        i = 0
        while i < len(tok):
            if i < len(tok) - 1 and (tok[i], tok[i + 1]) == s:
                new_tok.append(tok[i] + tok[i+1])
                i += 2
            else:
                new_tok.append(tok[i])
                i += 1
        new_enc[tuple(new_tok)] = new_enc.get(tuple(new_tok), 0) + freq
    enc = new_enc


In [150]:
enc

{(b'H', b'e', b'l', b'l', b'o'): 1,
 (b',',): 2,
 (b' ', b'w', b'or', b'l', b'd'): 1,
 (b'!',): 1,
 (b' ', b'I'): 1,
 (b"'", b'm'): 1,
 (b' ', b't', b'e', b's', b'ti', b'n', b'g'): 1,
 (b' ', b'G', b'P', b'T'): 1,
 (b'-',): 1,
 (b'2',): 1,
 (b"'", b's'): 1,
 (b' ', b'B', b'P', b'E'): 1,
 (b' ', b't', b'o', b'k', b'e', b'n', b'i', b'z', b'er'): 1,
 (b'.',): 2,
 (b'\n',): 2,
 (b'D', b'on'): 1,
 (b"'", b't'): 1,
 (b' ', b'sp', b'li', b't'): 1,
 (b' ', b'w', b'r', b'on', b'g', b'l', b'y'): 1,
 (b':',): 1,
 (b' ', b'n', b'u', b'm', b'b', b'er', b's'): 1,
 (b' ', b'li', b'k', b'e'): 1,
 (b' ', b'1', b'2', b'3'): 1,
 (b' ', b'p', b'u', b'n', b'c', b't', b'u', b'a', b'ti', b'on'): 1,
 (b'!', b'!', b'!'): 1,
 (b' ', b'or'): 1,
 (b' ', b'sp', b'a', b'c', b'e', b's'): 1,
 (b' ', b' '): 1,
 (b' ', b'm', b'a', b't', b't', b'er'): 1}

In [151]:
merges

[(b'o', b'n'),
 (b'e', b'r'),
 (b't', b'i'),
 (b's', b'p'),
 (b'o', b'r'),
 (b'l', b'i')]

In [153]:
import pickle
with open("pretokenized_counts.pkl", "rb") as f:
    fp = pickle.load(f)

In [173]:
enc = {}
for tok, freq in fp.items():
    bt = tuple(bytes([b]) for b in tok.encode("utf-8"))
    enc[bt] = enc.get(bt, 0) + freq
num_merges = 256
merges = []
d = {}
for e in enc:
    for i, j in zip(e[:-1], e[1:]):
        d[(i, j)] = d.get((i, j), 0) + enc[e]
for _ in range(num_merges):
    if not d:
        break
    s = max(d, key=lambda x: (d[x], x))
    # s = max(d, key = d.get)
    del d[s]
    merges.append(s)
    new_enc = {}
    for tok, freq in enc.items():
        new_tok = []
        i = 0
        while i < len(tok):
            if i < len(tok) - 1 and (tok[i], tok[i + 1]) == s:
                merged = tok[i] + tok[i + 1]
                if new_tok: # left neighbour, so decrement count for that and increase for new merged
                    old_leftn = (new_tok[-1], tok[i])
                    d[old_leftn] = d.get(old_leftn, 0) - freq
                    if d[old_leftn] <= 0:
                        d.pop(old_leftn, None)
                    new_ln = (new_tok[-1], merged)
                    d[new_ln] = d.get(new_ln, 0) + freq
                if i + 2 < len(tok): # right neighbour, so decrement count for that and increase for new merged
                    old_rightn = (tok[i + 1], tok[i + 2])
                    d[old_rightn] = d.get(old_rightn, 0) - freq
                    if d[old_rightn] <= 0:
                        d.pop(old_rightn, None)
                    new_rn = (merged, tok[i + 2])
                    d[new_rn] = d.get(new_rn, 0) + freq
                new_tok.append(merged)
                i += 2
            else:
                new_tok.append(tok[i])
                i += 1
        new_enc[tuple(new_tok)] = new_enc.get(tuple(new_tok), 0) + freq
    enc = new_enc


In [174]:
vocab = {bytes([i]) : i for i in range(256)}
for i, (a, b) in enumerate(merges):
    vocab[a + b] = i + 256


In [ ]:
def train_bpe(filepath: str, vocab_size: int = 512, special_tokens: list[str] = None) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    """Train a BPE tokenizer on a given text file.
       Inputs:
           filepath: path to the text file to train on
           vocab_size: desired vocabulary size (including initial byte vocab)
           special_tokens: list of special tokens to include in the vocabulary
       Returns:
           vocab: dictionary mapping token IDs to byte sequences
           merges: list of byte pair merges performed during training
    """
    # Read the file and get pretokenized counts
    with open(filepath, "rb") as f:
        fp = pickle.load(f)
    
    # Initialize encoding with byte pairs
    enc = {}
    for tok, freq in fp.items():
        bt = tuple(bytes([b]) for b in tok.encode("utf-8"))
        enc[bt] = enc.get(bt, 0) + freq
    
    # Perform BPE merges
    merges = []
    d = {}
    for e in enc:
        for i, j in zip(e[:-1], e[1:]):
            d[(i, j)] = d.get((i, j), 0) + enc[e]
    
    num_merges = vocab_size - 256  # Subtract initial byte vocabulary
    
    for _ in range(num_merges):
        if not d:
            break
        s = max(d, key=lambda x: (d[x], x))
        del d[s]
        merges.append(s)
        
        new_enc = {}
        for tok, freq in enc.items():
            new_tok = []
            i = 0
            while i < len(tok):
                if i < len(tok) - 1 and (tok[i], tok[i + 1]) == s:
                    merged = tok[i] + tok[i + 1]

                    if new_tok: # left neighbour, so decrement count for that and increase for new merged
                        old_leftn = (new_tok[-1], tok[i])
                        d[old_leftn] = d.get(old_leftn, 0) - freq
                        if d[old_leftn] <= 0:
                            d.pop(old_leftn, None)
                        new_ln = (new_tok[-1], merged)
                        d[new_ln] = d.get(new_ln, 0) + freq

                    if i + 2 < len(tok): # right neighbour, so decrement count for that and increase for new merged
                        old_rightn = (tok[i + 1], tok[i + 2])
                        d[old_rightn] = d.get(old_rightn, 0) - freq
                        if d[old_rightn] <= 0:
                            d.pop(old_rightn, None)
                        new_rn = (merged, tok[i + 2])
                        d[new_rn] = d.get(new_rn, 0) + freq
                    new_tok.append(merged)
                    i += 2
                else:
                    new_tok.append(tok[i])
                    i += 1
            new_enc[tuple(new_tok)] = new_enc.get(tuple(new_tok), 0) + freq
        enc = new_enc
        
    # Build vocabulary
    vocab = {i: bytes([i]) for i in range(256)}
    for i, (a, b) in enumerate(merges):
        vocab[i + 256] = a + b
    
    return vocab, merges


In [178]:
with open("bpe_vocab.pkl", "rb") as f:
    vocab = pickle.load(f)
with open("bpe_merges.pkl", "rb") as f:
    merges = pickle.load(f)

In [192]:
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [202]:
text = "the cat ate"
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
encs = [tuple(bytes([b]) for b in k.group(0).encode("utf-8")) for k in re.finditer(PAT, text)]
merge_ranks = {p: i for i, p in enumerate(merges)}
for i, enc in enumerate(encs):
    while True:
        pairs = [(enc[j], enc[j + 1]) for j in range(len(enc) - 1)]
        rankp = [(merge_ranks[p], j, p) for j, p in enumerate(pairs) if p in merge_ranks]
        if not rankp:
            break
        _, j, _ = min(rankp)
        enc = enc[:j] + (enc[j] + enc[j + 1],) + enc[j + 2:]
    encs[i] = enc


In [203]:
tok2id = {v: k for k, v in vocab.items()}

In [204]:
encoded = []
for i in range(len(encs)):
    encoded.extend([tok2id[x] for x in encs[i]])
encoded

[116, 258, 453, 451, 101]

In [205]:
def decode(encoded: list, vocab: dict[int, bytes]):
    s = b"".join([vocab[k] for k in encoded]).decode("utf-8")
    return s

In [207]:
decode(encoded, vocab)

'the cat ate'

In [208]:
from typing import Iterable, Iterator

class Tokenizer:
    def __init__(self, vocab, merges, special_tokens=None):
        self.vocab = vocab
        self.merges = merges
        self.special_tokens = special_tokens
        self.tok2id = {v: k for k, v in self.vocab.items()}
        self.merge_ranks = {p: i for i, p in enumerate(self.merges)}

    def encode(self, text: str) -> list[int]:
        PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        encs = [tuple(bytes([b]) for b in k.group(0).encode("utf-8")) for k in re.finditer(PAT, text)]
        for i, enc in enumerate(encs):
            while True:
                pairs = [(enc[j], enc[j + 1]) for j in range(len(enc) - 1)]
                rankp = [(self.merge_ranks[p], j, p) for j, p in enumerate(pairs) if p in self.merge_ranks]
                if not rankp:
                    break
                _, j, _ = min(rankp)
                enc = enc[:j] + (enc[j] + enc[j + 1],) + enc[j + 2:]
            encs[i] = enc
        encoded = []
        for i in range(len(encs)):
            encoded.extend([self.tok2id[x] for x in encs[i]])
        return encoded
    
    @classmethod
    def from_files(cls, vocab_filepath, merges_filepath, special_tokens=None):
        with open(vocab_filepath, "rb") as f:
            vocab = pickle.load(f)
        
        with open(merges_filepath, "rb") as f:
            merges = pickle.load(f)

        return cls(vocab=vocab, merges=merges, special_tokens=special_tokens)
    
    def decode(self, ids: list[int]) -> str:
       s = b"".join([self.vocab[k] for k in ids]).decode("utf-8")
       return s 
    
    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
        for s in iterable:
            enc = self.encode(s)
            for i in enc:
                yield i


In [ ]:
import torch
import torch.nn as nn
from einops import einsum
import math

class Linear(nn.Module):
    def __init__(self, in_features, out_features, device="cpu", dtype=None):
        super().__init__()
        self.device = device if device is not None else "cpu"
        self.W = nn.Parameter(torch.randn((out_features, in_features), dtype=dtype, device=device))
        sigma = math.sqrt(2 / (in_features + out_features))
        nn.init.trunc_normal_(self.W, mean=0, std=sigma, a=-3 * sigma, b=3 * sigma)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = einsum(x, self.W, "... i, o i -> ... o")
        return out
    
class Embedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, device=None, dtype=None):
        super().__init__()
        self.emb = nn.Parameter(torch.randn((num_embeddings, embedding_dim), device=device, dtype=dtype))
        nn.init.trunc_normal_(self.emb, mean=0, std=1, a=-3, b=3)

    def forward(self, token_ids: torch.Tensor) -> torch.Tensor:
        return self.emb[token_ids]
    
class RMSNorm(nn.Module):
    def __init__(self, d_model: int, eps: float = 1e-5, device=None, dtype=None):
        super().__init__()
        self.eps = eps
        self.d_model = d_model
        self.gamma = nn.Parameter(torch.eye(d_model, dtype=dtype, device=device))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        in_dtype = x.dtype
        x = x.to(torch.float32)
        rms = torch.sqrt(x.pow(2).mean(dim=-1, keepdim=True) + self.eps)
        out = (x @ self.gamma) / rms
        return out
    


In [2]:
!pip show torch

Name: torch
Version: 2.9.0+cu126
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org
Author: 
Author-email: PyTorch Team <packages@pytorch.org>
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvshmem-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision


In [1]:
from cs336_basics.train import *
from cs336_basics.layers import TransformerLM
from cs336_basics.tokenizer import Tokenizer
import torch

# vocab_path=r"D:\CS336\assignment1-basics\cs336_basics\bpe_vocab.pkl"
# merges_path=r"D:\CS336\assignment1-basics\cs336_basics\bpe_merges.pkl"
# tokenizer = Tokenizer.from_files(vocab_filepath=vocab_path, merges_filepath=merges_path)
# device = "cpu"
# model = TransformerLM(
#         vocab_size=tokenizer.vocab_size, 
#         context_length=128, 
#         num_layers=6,
#         d_model=256,
#         d_ff=1024,
#         num_heads=8,
#         theta=10000.0).to(device=device)
# # model = torch.compile(model)
# model_path = r"D:\CS336\assignment1-basics\ckpt_final.pt"
# d = torch.load(model_path)
# model.load_state_dict(d["model"])

# prompt = "What are you doing?"
# print("Prompt length:", len(tokenizer.encode(prompt)))
# output = generate(model, tokenizer, prompt, 128)
# print(output)

ModuleNotFoundError: No module named 'cs336_basics'

In [21]:
# Number of parameter tensors:
# len(list(model.parameters()))

# Total number of scalar parameters:
sum(p.numel() for p in model.parameters())

6556928

In [15]:
prompt = "Happy birthday"
output = generate(model, tokenizer, prompt, 128, temperature=1)
print(output)

Happy birthday they thinys it of sandglve soitmot from up! Tinow ligh up on their fun, unts Lily for comdar that another wind mom delicse beet shy wore asle gifGark from and as lat mom shoot pill under on them!" Ar went goit from them under The wainan uninor." Tim ble happy hot napl of j


In [3]:
tokenizer.vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [6]:
!nvidia-smi

Tue Dec 30 13:08:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----